In [ ]:
import pandas as pd
df_train=pd.read_csv('train (2).csv')
df_val=pd.read_csv('val (4).csv')
df_test=pd.read_csv('Task2_ES_test_no_label (1).csv')

In [ ]:
df_train.head()

,text,binary,multiclass,id
0,Mientras me persigno y le rezo a la Virgen del...,Not Hope,Not Hope,5533
1,No.. Yo ya no estoy para esperar.. Ni para rog...,Not Hope,Not Hope,549
2,Creo que estamos ante el mejor episodio de est...,Hope,Generalized Hope,4350
3,Bueno ojalá q cuando llegue este fin de semana...,Hope,Realistic Hope,3593
4,#USER# #USER# Marcos de boludo no tiene un pel...,Not Hope,Not Hope,770


In [ ]:
df_train['multiclass'].value_counts()

Not Hope            4701
Generalized Hope    1151
Unrealistic Hope     546
Realistic Hope       505
Name: multiclass, dtype: int64

In [ ]:
!pip install num2words

In [ ]:
import re
import string
import emoji
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import contractions

def preprocess(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Convert emojis to textual representation
    text = emoji.demojize(text, delimiters=(" ", " "))
    text = text.replace("_", "  ")

    # Remove special characters and punctuation
    exclude = set(string.punctuation + '🏳️‍⚧️🔍¿')
    text = ''.join(ch for ch in text if ch not in exclude)

    # Fix contractions
    text = contractions.fix(text, slang=True)

    # Remove usernames
    text = re.sub('@[a-zA-Z0-9_]*', '', text)

    # Replace '$' with 'dollar'
    text = text.replace('$', ' dollar ')

    # Convert text to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    english_stopwords = stopwords.words('spanish')
    text = [t for t in tokens if t not in english_stopwords]
    text = " ".join(text)

    return text


In [ ]:
!pip install inflect

In [ ]:
import inflect
p = inflect.engine()

In [ ]:
def number_to_words(text):
    return ' '.join(p.number_to_words(token) if token.isdigit() else token for token in text.split())

In [ ]:
from nltk.corpus import stopwords, wordnet

import emoji
#import demoji
#demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

In [ ]:
df_train['clean_text'] = df_train['text'].apply(lambda x:emo(x))
#df_train["clean_text"]=df_train['clean_text'].apply(lambda X: number_to_words(X))
df_train["clean_text"]=df_train['clean_text'].apply(lambda X: preprocess(X))

df_val['clean_text'] = df_val['text'].apply(lambda x:emo(x))
#df_val["clean_text"]=df_val['clean_text'].apply(lambda X: number_to_words(X))
df_val["clean_text"]=df_val['clean_text'].apply(lambda X: preprocess(X))


df_test['clean_text'] = df_test['text'].apply(lambda x:emo(x))
#df_test["clean_text"]=df_test['clean_text'].apply(lambda X: number_to_words(X))
df_test["clean_text"]=df_test['clean_text'].apply(lambda X: preprocess(X))

In [ ]:
x_train=df_train['clean_text'].tolist()

In [ ]:
x_test=df_test['clean_text'].tolist()

In [ ]:
x_val=df_val['clean_text'].tolist()

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train['multiclass'])
y_val =label_encoder.transform(df_val['multiclass'])

In [ ]:
import numpy as np
from tqdm import tqdm #dont run this
benvocab = []
embedding_index_ben = {}
f = open('wiki.es.align.vec',encoding='utf-8')
for line in f:
        values = line.rstrip().split(' ')
        word = values[0]
        benvocab.append(word)
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index_ben[word] = coefs

In [ ]:
def get_vectors(sentence):# dont run
    sentvec = np.zeros(300)
    zeros = np.zeros(300)

    for word in sentence.split():
        if word in benvocab:
             temp = embedding_index_ben[word]
        else: temp = zeros
        # print('word : ', word, ' ttt : ', temp)
        sentvec += temp

    sentvec = sentvec / len(sentence.split())
    return sentvec

In [ ]:
# X=pd.DataFrame(x)
# Y=pd.DataFrame(y)

# # X_test11=pd.DataFrame(x_test)
# # y_test11=pd.DataFrame(y_test)

In [ ]:
# X=X.rename(columns={0:'text'})

In [ ]:
# Y=Y.rename(columns={0:'label'})
# X_test11=X_test11.rename(columns={0:'text'})
# y_test11=y_test11.rename(columns={0:'label'})

In [ ]:
# df=pd.concat([X,Y], axis=1)

In [ ]:
#df_train=pd.concat([X_train11,y_train11], axis=1)

In [ ]:
# df

In [ ]:
df_train['vec'] = df_train['clean_text'].apply(lambda x:get_vectors(x))#dont run this
df_val['vec'] = df_val['clean_text'].apply(lambda x:get_vectors(x))
df_test['vec'] = df_test['clean_text'].apply(lambda x:get_vectors(x))

In [ ]:
# import pickle #dont run this
# with open('/content/drive/MyDrive/vectrain_file.pkl', 'wb') as f:  # open a text file
#     pickle.dump(df['vec'], f) # serialize the list

In [ ]:
#xtest = df_test['text'].apply(lambda x:get_vectors(x))

In [ ]:
# import pickle
# with open('/content/drive/MyDrive/vectrain_file.pkl', 'rb') as f:

#     train_loaded = pickle.load(f) # deserialize using load()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC

In [ ]:
classifiers=[]

In [ ]:
SVM = svm.SVC(probability=True, kernel="rbf", class_weight="balanced")
classifiers.append(SVM)
svc =  LinearSVC(class_weight='balanced')
classifiers.append(svc)
RFC = RandomForestClassifier(n_estimators=200, random_state=0)
classifiers.append(RFC)
LR = LogisticRegression(random_state=0, solver='lbfgs',
                            multi_class='multinomial')
classifiers.append(LR)
MLP = MLPClassifier()
classifiers.append(MLP)
kNNeigh = KNeighborsClassifier()
classifiers.append(kNNeigh)

In [ ]:
# # MNB = MultinomialNB()
# # classifiers.append(MNB)
# gbc = GradientBoostingClassifier()
# classifiers.append(gbc)

# classifiers.append(gbc)
# SVM = svm.SVC()
# classifiers.append(SVM)
# DT = tree.DecisionTreeClassifier()
# classifiers.append(DT)
# RFC = RandomForestClassifier()
# classifiers.append(RFC)
# LR = LogisticRegression()
# classifiers.append(LR)
# #MLP = MLPClassifier(random_state=1, max_iter=300)
# # MLP = MLPClassifier()
# # classifiers.append(MLP)
# Percpt= Perceptron()
# classifiers.append(Percpt)
# # svc=SVC()
# # classifiers.append(svc)
# kNNeigh = KNeighborsClassifier()#p:n_neighbors=3
# classifiers.append(kNNeigh)
# xgb=XGBClassifier()
# classifiers.append(xgb)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
for clf in classifiers:
    clf.fit(df_train['vec'].tolist(), y_train.tolist())
    y_pred= clf.predict(df_val['vec'].tolist())
    acc = accuracy_score(y_val.tolist(), y_pred)
    print("Accuracy of %s is %s"%(clf, acc))
    cm = confusion_matrix(y_val.tolist(), y_pred)
    print("Confusion Matrix of %s is %s"%(clf, cm))
    report = classification_report(y_val.tolist(), y_pred)
    print("Classificationreport is  of %s is %s"%(clf, report))

Accuracy of SVC(class_weight='balanced', probability=True) is 0.6060869565217392
Confusion Matrix of SVC(class_weight='balanced', probability=True) is [[103  42  21  20]
 [107 531  64  97]
 [ 14  26  23  11]
 [ 14  19  18  40]]
Classificationreport is  of SVC(class_weight='balanced', probability=True) is               precision    recall  f1-score   support

           0       0.43      0.55      0.49       186
           1       0.86      0.66      0.75       799
           2       0.18      0.31      0.23        74
           3       0.24      0.44      0.31        91

    accuracy                           0.61      1150
   macro avg       0.43      0.49      0.44      1150
weighted avg       0.70      0.61      0.64      1150

Accuracy of LinearSVC(class_weight='balanced') is 0.6947826086956522
Confusion Matrix of LinearSVC(class_weight='balanced') is [[ 95  67   9  15]
 [ 72 661  28  38]
 [ 12  40  14   8]
 [  9  37  16  29]]
Classificationreport is  of LinearSVC(class_weight='bal

C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\linear_model

Accuracy of LogisticRegression(multi_class='multinomial', random_state=0) is 0.7234782608695652
Confusion Matrix of LogisticRegression(multi_class='multinomial', random_state=0) is [[ 50 135   0   1]
 [ 20 779   0   0]
 [ 11  62   0   1]
 [  4  84   0   3]]
Classificationreport is  of LogisticRegression(multi_class='multinomial', random_state=0) is               precision    recall  f1-score   support

           0       0.59      0.27      0.37       186
           1       0.73      0.97      0.84       799
           2       0.00      0.00      0.00        74
           3       0.60      0.03      0.06        91

    accuracy                           0.72      1150
   macro avg       0.48      0.32      0.32      1150
weighted avg       0.65      0.72      0.65      1150

Accuracy of MLPClassifier() is 0.7104347826086956
Confusion Matrix of MLPClassifier() is [[ 82  88   8   8]
 [ 60 708  14  17]
 [ 16  39   9  10]
 [  7  59   7  18]]
Classificationreport is  of MLPClassifier() is  

C:\Users\CCL\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\CCL\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [ ]:
svc.fit(df_train['vec'].tolist(), y_train.tolist())
y_pred_test= svc.predict(df_test['vec'].tolist())

In [ ]:
predct_Test=label_encoder.inverse_transform(y_pred_test)

In [ ]:
predct_Test

array(['Not Hope', 'Not Hope', 'Not Hope', ..., 'Not Hope', 'Not Hope',
       'Not Hope'], dtype=object)

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels with CTbert
pred_testk = pd.DataFrame(data=predct_Test, columns=['Prediction'])
submission_t1span= pd.DataFrame()
submission_t1span['id'] = df_test['id']
submission_t1span['multiclass'] = pred_testk
submission_t1span.to_csv('task2_span_Withoutoversample_aignvec_svc_46.csv')